# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import time 

import gmaps.datasets

#from ipywidgets.embed import embed_minimal_html 


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "../clean_city_data.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,61.05,92,100,20.42,AU,1622218874
1,Karwar,14.8000,74.1333,81.77,78,19,6.06,IN,1622218877
2,Yeppoon,-23.1333,150.7333,49.78,50,25,7.29,AU,1622218594
3,New Norfolk,-42.7826,147.0587,45.90,67,22,1.99,AU,1622218882
4,Longyearbyen,78.2186,15.6401,31.84,59,0,17.27,SJ,1622218884
...,...,...,...,...,...,...,...,...,...
545,Sur,22.5667,59.5289,88.29,43,0,9.84,OM,1622220170
546,Bowen,-20.0167,148.2333,48.42,49,0,8.23,AU,1622220172
547,Aksarka,66.5606,67.7975,44.67,43,90,6.82,RU,1622220174
548,Kulykivka,51.3735,31.6466,65.57,81,92,1.57,UA,1622220177


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [9]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [12]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Note:
> I included 2 screenshots of the heat maps in the images folder to make sure its the required angle.
***

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
424,Maun,-19.9833,23.4167,71.56,33,0,3.44,BW,1622219884
434,Ploemeur,47.7333,-3.4333,72.75,50,0,9.22,FR,1622219907
436,Saint-Pierre,-21.3393,55.4781,73.04,78,0,4.61,RE,1622219618
488,San Miguel de Tucumán,-26.8241,-65.2226,73.40,56,0,3.36,AR,1622219966
534,Abhā,18.2164,42.5053,73.22,49,0,4.90,SA,1622220129


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

Issues with finding a hotel at Maun
Issues with finding a hotel at Ploemeur
Issues with finding a hotel at Saint-Pierre
Issues with finding a hotel at San Miguel de Tucumán
Issues with finding a hotel at Abhā


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
424,Maun,-19.9833,23.4167,71.56,33,0,3.44,BW,1622219884,NA
434,Ploemeur,47.7333,-3.4333,72.75,50,0,9.22,FR,1622219907,NA
436,Saint-Pierre,-21.3393,55.4781,73.04,78,0,4.61,RE,1622219618,NA
488,San Miguel de Tucumán,-26.8241,-65.2226,73.40,56,0,3.36,AR,1622219966,NA
534,Abhā,18.2164,42.5053,73.22,49,0,4.90,SA,1622220129,NA


In [15]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [16]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

0

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Checking hotel info:
len(hotel_info)

0

In [19]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Note:
> I included 2 screenshots of the hotel maps in the images folder to make sure its the required angle. 
***